Lab: Pipelines & Grid Search with scikit-learn
====

By The End Of This Lab You Should Be Able To:
----

- Define a Pipeline to fit several ML algorithms at a time 
- Perform Grid Search to improve performance of algorithm

In [1]:
reset -fs

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

import warnings
warnings.filterwarnings('ignore')

palette = "Dark2"
%matplotlib inline

In [3]:
from sklearn.datasets import make_moons

In [ ]:
noisy_moons = make_moons(n_samples=10_000, noise=.15, random_state=42)

sns.scatterplot(x=noisy_moons[0][:, 0],
                y=noisy_moons[0][:, 1],
                hue=noisy_moons[1]);

In [151]:
from sklearn.model_selection import train_test_split

# Load and split the data
X_train, X_test, y_train, y_test = train_test_split(noisy_moons[0], noisy_moons[1], test_size=0.3, random_state=42)

In [152]:
from sklearn.pipeline import Pipeline

In [153]:
from typing import List

def make_pipelines() -> List[Pipeline]:
    """Create a pipeline for each of the following algorithms:
    1. Logistic Regression
    2. k-NN
    3. Naive Bayes (Guassian)
    4. SVM
    5. Decision Tree
    
    Use default hyperparameters, expect if an algorithm takes random_state then random_state=42 
    
    Return a list of all the pipelines.
    """ 
    from sklearn.linear_model import LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.svm import SVC
    from sklearn.tree import DecisionTreeClassifier
    pipelines = []
    pipelines.append(Pipeline([('clf', LogisticRegression())]))
    pipelines.append(Pipeline([('clf', KNeighborsClassifier())]))
    pipelines.append(Pipeline([('clf', GaussianNB())]))
    pipelines.append(Pipeline([('clf', SVC())]))
    pipelines.append(Pipeline([('clf', DecisionTreeClassifier())]))
    
    
    # YOUR CODE HERE
    # raise NotImplementedError()
    return pipelines

In [154]:
"""
3 points
Test code for the 'make_pipelines' function. 
This cell should NOT give any errors when it is run.
"""

pipelines = make_pipelines()

assert len(pipelines) == 5
assert all(type(pipe) == sklearn.pipeline.Pipeline for pipe in pipelines)
assert all(len(pipe.steps) == 1 for pipe in pipelines)

In [155]:
def fit_pipelines(pipelines: List[Pipeline]):
    """Fit each pipeline to the training data. 
    The function will automatically update the global state of each pipeline, thus needs to return nothing
    """
    for pipe in pipelines:
        pipe.fit(X_train, y_train)
    
fit_pipelines(pipelines)

In [157]:
def order_models_moon_data() -> List[str]:
    """Order models from best to worst on `score`. 
    Create the list either manually or write code.
    Return a list of names: [..., GaussianNB]
    
    For informational purposes, list of scores: [0.9923, 0.9900,  0.9863, 0.8853, 0.8840]
    Thus, GaussianNB scored the worst at 0.8840
    """
    models_best_to_worst = [(pipe.steps[0][1].__class__.__name__, pipe.score(X_test, y_test)) \
                            for pipe in pipelines]
    models_best_to_worst.sort(key=lambda x: x[1], reverse=True)
    models_best_to_worst = [x[0] for x in models_best_to_worst]
    # YOUR CODE HERE
    # raise NotImplementedError()
    return models_best_to_worst

In [158]:
"""
2 points
Test code for the 'order_models_moon_data' function. 
NOTE: There are hidden tests. They will make sure all the models are in order
This cell should NOT give any errors when it is run.
"""

models_best_to_worst = order_models_moon_data()

assert len(models_best_to_worst) == 5
assert models_best_to_worst[-1] == 'GaussianNB'


------
Glass dataset
----

6 types of glass

Defined in terms of their oxide content (i.e. Na, Fe, K, etc)

> The study of classification of types of glass was motivated by criminological investigation. At the scene of the crime, the glass left can be used as evidence...if it is correctly identified!

Read more: https://archive.ics.uci.edu/ml/datasets/glass+identification

In [159]:
# Hint
import numpy as np

In [160]:
def load_csv_into_numpy():
    "Load csv, then split into data and targets"
    df = pd.read_csv('glass.csv')
    X = df.drop(columns='Type')
    X= np.array(X)
    y = df['Type']
    y = np.array(y)
    # YOUR CODE HERE
    # raise NotImplementedError()
    return X, y

In [161]:
"""
3 points
Test code for the 'load_csv_into_numpy' function. 
This cell should NOT give any errors when it is run.
"""

X, y = load_csv_into_numpy()

assert type(X) == np.ndarray
assert X.shape == (214, 9)
assert list(X[-1]) == [1.51711, 14.23,  0.,  2.08, 73.36,  0., 8.62,  1.67,  0.]

assert type(y) == np.ndarray
assert y.shape == (214,)
assert list(np.unique(y)) == [1.0, 2.0, 3.0, 5.0, 6.0, 7.0]
assert y[-1] == 7

In [162]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [163]:
# We get to reuse our earlier code. Functions are 🆒
pipelines = make_pipelines()
fit_pipelines(pipelines)

In [165]:
def order_models_glass_data() -> List[str]:
    """Order models from best to worst on `score`. 
    Create the list manually  or write code.
    Return a list of names
    
    For informational purposes, list of scores: [0.68519, 0.64815, 0.62963, 0.59259, 0.35185] 

    """
    models_best_to_worst = [(pipe.steps[0][1].__class__.__name__, pipe.score(X_test, y_test)) \
                            for pipe in pipelines]
    models_best_to_worst.sort(key=lambda x: x[1], reverse=True)
    models_best_to_worst = [x[0] for x in models_best_to_worst]
    

    # YOUR CODE HERE
    # raise NotImplementedError()
    return models_best_to_worst

In [166]:
"""
2 points
Test code for the 'order_models' function. 
There are hidden tests. They will make sure all the models are in order
This cell should NOT give any errors when it is run.
"""

models_best_to_worst = order_models_glass_data()
assert len(models_best_to_worst) == 5

In [167]:
models_best_to_worst

['DecisionTreeClassifier',
 'SVC',
 'KNeighborsClassifier',
 'LogisticRegression',
 'GaussianNB']

Let's improve the KNeighborsClassifier for the glass data with Grid Search
-----

In [168]:
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [193]:
def grid_search_knn():
    """
    1. Create a pipeline with PCA into k-NN.
    2. Define a grid search for n_components of PCA and n_neighbors of k-NN
    3. Conduct grid search
    4. Return score (accuracy) and dictionary of best parameters
    """
    pipeline_knn = Pipeline([('pca', PCA()),
                         ('clf', KNeighborsClassifier())])
    
    grid_params = dict(pca__n_components=range(1,10),
                       clf__n_neighbors=range(1,10))
    
    gs = GridSearchCV(estimator = pipeline_knn,
                      param_grid =  grid_params,
                      scoring = 'accuracy')
    gs.fit(X_train, y_train)
    accuracy = gs.score(X_test, y_test)
    best_params = gs.best_params_
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    return accuracy, best_params

In [194]:
"""
8 points
Test code for the 'grid_search_knn' function. 
There are hidden tests. 
This cell should NOT give any errors when it is run.
"""
accuracy, best_params = grid_search_knn()

assert accuracy == 0.7407407407407407
assert list(best_params.keys()) == ['clf__n_neighbors', 'pca__n_components']


Bonus Cartoon
------

<center><img src="https://imgs.xkcd.com/comics/data_pipeline.png" width="75%"/></center>

> "Is the pipeline literally running from your laptop?"   
> "Don't be silly, my laptop disconnects far too often to host a service we rely on. It's running on my phone."


<br>
<br> 
<br>

----